# Capstone - Toronto

## Load Toronto data from Wikipedia

Install lxml required by pandas read_html method

In [7]:
!conda install -c conda-forge lxml --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libxslt-1.1.33             |       h7d1a2b0_0         426 KB
    lxml-3.8.0                 |           py36_0         3.8 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.2 MB

The following NEW packages will be INSTALLED:

  libxslt            pkgs/main/linux-64::libxslt-1.1.33-h7d1a2b0_0
  lxml               conda-forge/linux-64::lxml-3.8.0-py36_0



lxml-3.8.0           | 3.8 MB    | ##################################### | 100% 
libxslt-1.1.33       | 426 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [8]:
import pandas as pd
import numpy as np
from lxml import etree
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# read first table in url
dft = pd.read_html(url)[0]
# drop rows with "Not assigned" postal codes
dft.drop(index=dft[dft['Borough'] == "Not assigned"].index, inplace=True)
# reset index after dropping rows
dft.reset_index(drop=True, inplace=True)
# replace NaN in 'Neighborhood' column with value from 'Borough' column
# first method
#dft.loc[dft['Neighborhood'] == np.nan, 'Neighborhood'] = dft['Borough']
# second method
dft['Neighborhood'].replace(np.nan, dft['Borough'], inplace=True)
# check if missing data exists
missing_data = dft.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("") 
# display dataframe
dft

Postal Code
False    103
Name: Postal Code, dtype: int64

Borough
False    103
Name: Borough, dtype: int64

Neighborhood
False    103
Name: Neighborhood, dtype: int64



,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing Centre
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Display Toronto dataframe shape

In [9]:
dft.shape

(103, 3)

## Find latitude and longitude

In [10]:
import geocoder # import geocoder

def find_lat_lng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return [latitude, longitude]

ModuleNotFoundError: No module named 'geocoder'

In [11]:
Use geocoder to find

SyntaxError: invalid syntax (<ipython-input-11-3f5d70bf0779>, line 1)

In [12]:
!wget -q -O 'toronto.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [13]:
toronto_df = pd.read_csv('toronto.csv')
toronto_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
df_ll = pd.merge(dft[['Postal Code', 'Borough', 'Neighborhood']], toronto_df, on = 'Postal Code', how = 'outer')
df_ll.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
